In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki.name

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(web_path=["https://docs.langchain.com"])
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
#retriever

# Convert the retriever into a tool
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "Langsmith_search", "Search for information about Langsmith. For any questions about Langsmith, you must use this tool")
retriever_tool.name


In [ ]:
# Arxiv Tool [Where all research paper lies] 
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

In [ ]:
tools = [wiki, arxiv, retriever_tool]
#tools

In [13]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

In [ ]:
# Optional [Not necessary]
from langsmith import Client

client = Client()
prompt = client.pull_prompt("hwchase17/openai-functions-agent")
print(getattr(prompt, "messages", None))


In [ ]:
# Agents
from langchain.agents import create_agent
from langchain_core import messages

system_prompt_text = """
You are a concise assistant. 
Give short, clear answers with NO extra formatting, NO explanations about your reasoning, 
NO step-by-step text.
Just answer the question directly in 5–8 sentences.
"""

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt_text
)

# Beautify the Output
def beautify_response(response):
    messages = response["messages"]

    for m in messages:
        # Determine label based on message type
        role = m.type.upper()

        print(f"🔹 {role} MESSAGE")
        print("----------------------------")

        # For assistant messages that have tool calls
        if hasattr(m, "tool_calls") and m.tool_calls:
            print("Tool Calls:")
            for tc in m.tool_calls:
                print(f"  - Tool: {tc.get('name')}")
                print(f"    Args: {tc.get('args')}")
                print()

        # Tool output (ToolMessage)
        if m.type == "tool":
            print(f"Tool Name: {m.name}")
            print("Output:")
            print(m.content)
            print()
            continue

        # Regular content (Human + AI)
        print(m.content)
        print("\n")

response = agent.invoke({"messages": [{"role": "user", "content": "Who is Hrithik Roshan"}]})
beautify_response(response)
